# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [1]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score

import operator
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [3]:
# Tworymy wszystkie możliwe kombnacje 3 klasyfikatorów spośród podanych
# .fit(data_set, labels)
lr = LinearRegression()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier()
svc = SVC(gamma = 'auto')
nb = GaussianNB()
qda = QuadraticDiscriminantAnalysis()
names = [lr,dt,knn,svc,nb,qda]

comb_class = list(combinations(names,3))

def build_classifiers(komb):
    
    # fill this part
    c1 = komb[0].fit(data_set, labels)
    c2 = komb[1].fit(data_set, labels)
    c3 = komb[2].fit(data_set, labels)
    
    return c1, c2, c3

In [4]:
def build_stacked_classifier(classifiers,k):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    dt = DecisionTreeClassifier() # działa dobrze - bo ten klasyfikator
    knn = KNeighborsClassifier() # wszędzi accuracy = 0.0
    svc = SVC() # działa
    nb = GaussianNB() # wszędzie accuracy = 1.0
    qda = QuadraticDiscriminantAnalysis() # działa dobrze
    
    stacked_classifier =  k # set here
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [5]:
stack = [DecisionTreeClassifier(),KNeighborsClassifier(),SVC(gamma = 'auto'),GaussianNB(),QuadraticDiscriminantAnalysis()]
# w stacks
# Nie używamy LinearRegression, ponieważ "Classification metrics can't handle a mix of binary and continuous targets"
acc = []
maxy = []
for i in range(len(stack)):
    acc_3comb = []
    for comb in comb_class:
        classifiers = build_classifiers(comb)
        predicted = build_stacked_classifier(classifiers, stack[i])
        accuracy = accuracy_score(test_labels, predicted)
        acc_3comb.append(accuracy)
        
    index, value = max(enumerate(acc_3comb), key=operator.itemgetter(1))
    maxy.append([index,value])
    acc.append(acc_3comb)
maxy

/home/user/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/user/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/user/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/user/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


[[12, 0.95], [0, 0.0], [12, 0.85], [0, 1.0], [5, 0.95]]

In [6]:
# jak widać to która 3-ka klasfikatorów w pierwszej warstwie będzie najlepsza zależy od klasyfikatora
# użytego w drugiej warstwie
# Jednak dla dwóch różnych klasyfikatorów wartswy drugiej (dt,svc) najlepsze prognozy daje użycie w 1. warstwie :
comb_class[12] #(dt, knn, qda)

(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=None, n_neighbors=5, p=2,
            weights='uniform'),
 QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                store_covariance=False, store_covariances=None, tol=0.0001))

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [7]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

# wydaje mi się, że ta funkcja jest konieczna
# za względu na podany wzór:
def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result


def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    #print(Z)
    return (1+Z)/1.0

# Poprzednio error rate był nam potrzebny do licznia alf- do czego jest teraz ?
#def calculate_error(weights, model):
#    predicted = model.predict(test_set)
#    return np.dot(weights,calculate_accuracy_vector(predicted, test_labels))

Fill the two functions below:

In [8]:
def set_new_weights(model): #def set_new_weights(old_weights, alpha, model): tu nam zbędne stare wagi i alfy
    # fill the code here (two lines)
    # Zakładam, że 1 w podanym wzorze to wektor samych 1-ek
    # 1000 = len(calculate_vector-model(...))
    
                                            # calculate_accuracy_vector(predicted, labels)
    weights_numerator = calculate_accuracy_vector(model.predict(test_set), test_labels)
    new_weights = [x+1 for x in weights_numerator]
    return new_weights/(np.sum(new_weights))


#def calculate_error(model):
    # fill the code here (two lines)
#    pass


# PYTANIEA:
# Czy w takim razie usunąć funkcję calculate_error(model) z poprzeniej komórki?
# Czy wogóle nie liczyć alf - bo nie będą potrzebne przy predykcji przez głosowanie, a jeśli tak, to 
# funkcja calculate_error(model) staje się zbędna, ponieważ error_rate służył tylko do wyliczenia alf?

In [9]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
#alphas = [] - ?
classifiers = []
for iteration in range(number_of_iterations):
    
    #error_rate = calculate_error( model) # ta linijka jest potrzebna tylko do dalszego wylicznia alf
    #alpha = set_alpha(error_rate)
    
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00065531 0.00131062 0.00131062 0.00131062 0.00065531 0.00131062
 0.00131062 0.00131062 0.00131062 0.00131062 0.00065531 0.00131062
 0.00131062 0.00131062 0.00131062 0.00131062 0.00065531 0.00065531
 0.00131062 0.00131062 0.00131062 0.00065531 0.00131062 0.00131062
 0.00065531 0.00065531 0.00065531 0.00131062 0.00131062 0.00065531
 0.00065531 0.00131062 0.00131062 0.00131062 0.00131062 0.00131062
 0.00131062 0.00131062 0.00065531 0.00131062 0.00131062 0.00065531
 0.00131062 0.00131062 0.00131062 0.00131062 0.00131062 0.00131062
 0.00065531 0.00131062 0.00131062 0.00065531 0.00131062 0.00065531
 0.00131062 0.00065531 0.00065531 0.00131062 0.00065531 0.00131062
 0.00131062 0.00131062 0.00065531 0.00131062 0.00131062 0.00131062
 0.00131062 0.00131062 0.00131062 0.00131062 0.00065531 0.00131062
 0.00131062 0.00065531 0.00131062 0.00131062 0.00131062 0.00065531
 0.00131062 0.00065531 0.00065531 0.00065531 0.00131062 0.00131062
 0.00131062 0.00131062 0.00131062 0.00131062 0.00131062 0.0013

Train the classifier with the code below:

Set the validation data set:

In [10]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [11]:
def get_prediction(x):
    """
    the prediction is done with a voting method.
    """
    # fill the code here (5-6 lines)
    predictions = []
    for i in range(len(classifiers)):
        predictions.append(classifiers[i].predict(x)[0]) # dla różnie wytrnowanych klasyfikatorów - różne przewidywania
    labels_count = {}
    for label in np.unique(predictions):
        labels_count[label] = predictions.count(label)

    return max(labels_count.items(), key=operator.itemgetter(1))[0]

Test it:

In [12]:
prediction = get_prediction(validate_x)

print(prediction)

0
